f# Data Exploration

In [2]:
import pandas as pd
# import swifter

import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
from scipy.stats import ttest_ind, mannwhitneyu, kstest

from rapidfuzz import process

In [11]:
%%time

# data = pd.read_parquet("../data/interim/chrome/08_12_2022/http.0.parquet.gzip")
# data1 = pd.read_parquet("../data/interim/chrome/08_12_2022/http.1.parquet.gzip")
# data2 = pd.read_parquet("../data/interim/chrome/08_12_2022/http.2.parquet.gzip")
# data3 = pd.read_parquet("../data/interim/chrome/08_12_2022/http.3.parquet.gzip")
# data4 = pd.read_parquet("../data/interim/chrome/08_12_2022/http.4.parquet.gzip")
# data5 = pd.read_parquet("../data/interim/chrome/08_12_2022/http.5.parquet.gzip")

data_train = pd.read_parquet(
    "../data/processed/chrome/08_12_2022/train_set_01.parquet.gzip",
    engine="pyarrow",
    dtype_backend='pyarrow',
)

# data_test = pd.read_parquet(
#     "../data/processed/chrome/08_12_2022/test_set_01.parquet.gzip"
# )

In [12]:
data_train.dtypes

In [14]:
data_train.info(memory_usage="deep")

In [5]:
data = pd.concat([data_train, data_test])

In [229]:
# pandarallel.initialize(progress_bar=True)

In [22]:
%%time
data_train

In [4]:
data.info(memory_usage="deep")

# 14GB -> 763MB

In [8]:
data.dtypes

### Exploration with New Datatypes

In [116]:
(data.isna().sum() / len(data)) >= 0.95

### Check Tracking Classification and Create New Label Column

In [270]:
data["easylist"] = data["easylist"].astype(np.int32)
data["easyprivacy"] = data["easyprivacy"].astype(np.int32)

In [9]:
sns.histplot(data, x="easylist")
sns.displot(data, x="easyprivacy")

In [271]:
data["tracker"] = np.where(
    np.logical_or(data.easylist == 1, data.easyprivacy == 1), 1, 0
)
data["tracker"] = data["tracker"].astype(np.int32)

In [272]:
%%time
data.drop(["easylist", "easyprivacy"], axis=1, inplace=True)

In [4]:
plt.figure(figsize=(14, 7))
sns.histplot(data, x="tracker")

In [295]:
data['tracker'].value_counts()

class_0 = len(data[data['tracker'] == 0]) / 10000
class_1 = len(data[data['tracker'] == 1]) / 10000

class_distribution_ratio = class_1 / class_0
print(class_distribution_ratio)
print(class_0, class_1)

#### Remove empty columns

In [13]:
%%time
empty_columns = [col for col in data if data[col].isnull().all() == True]

In [14]:
data.drop(empty_columns, axis = 1, inplace = True)

### Put tracker col as last column

In [32]:
# https://sparkbyexamples.com/pandas/pandas-change-position-of-a-column/
temp_cols = data.columns.tolist()
index_col = data.columns.get_loc("tracker")
new_col_order = temp_cols[0:index_col] + temp_cols[index_col+1:] + temp_cols[index_col:index_col+1]

data = data[new_col_order]

#### Remove duplicate columns

In [81]:
data = data[~data.iloc[:, 6:-2].duplicated(keep="first")].reset_index(drop=True)

#### Add #Values per HTTP Message

In [17]:
%%time
data['header_count'] = data.iloc[:,6:-1].notnull().sum(axis=1)

### Check Column Name Similarity

In [57]:
def new_fuzzy_string_matching_for_column(col_name, col_values):
    result = pd.DataFrame(
        process.extract(
            col_name, col_values, processor=None, score_cutoff=80, limit=100
        ),
        columns=["fuzzy_match", "w_ratio", "index"],
    )
    result["col_name"] = col_name
    return result

In [59]:
%%time
data_column_values = data.columns.values[6:-2].tolist()

match = [
    new_fuzzy_string_matching_for_column(j, data_column_values[i + 1 :])
    for i, j in enumerate(data_column_values)
    if i != len(data_column_values) - 1
]

match2 = pd.concat(match, ignore_index=True)
print(match2)

Indices from match2 are wrong

In [9]:
match2[match2['col_name'] == 'content-length']

In [26]:
from scipy.special import comb

In [55]:
import sys
sys.setrecursionlimit(100000)

In [48]:
def recursive(a, b):
    if a == 1:
        return b
    else:
        # print(a, b, comb(a, 2))
        a = a - 1
        b = b + a
        # print(a, b)
        return recursive(a, b)

In [46]:
comb(5316,2)

In [60]:
def find_cols_with_similar_values(fuzzy_match, column):
    value_fuzzy = set(data[fuzzy_match].values)
    value_column = set(data[column].values)

    try:
        value_fuzzy.remove(None)
        value_column.remove(None)
    except KeyError:
        pass

    if (len([True for i in value_fuzzy if i in value_column]) / len(value_fuzzy)) > 0.5:
        return fuzzy_match, column
    else:
        return None

In [61]:
def select_similar_columns(fuzzy_match, column):
    row = match2.loc[(match2['fuzzy_match'] == fuzzy_match) & (match2['col_name'] == column)]
    index = row.index[0]
    match2.drop(index, inplace=True)
    return row

In [62]:
%%time
result = [find_cols_with_similar_values(col, col2) for col, col2 in zip(match2['fuzzy_match'], match2['col_name'])]
result

In [17]:
similar_values = [select_similar_columns(col[0], col[1]) for col in result if col is not None]

In [18]:
similar_values = pd.concat(similar_values, ignore_index=True)

In [19]:
similar_values

In [20]:
data.reset_index(drop=True, inplace=True)

In [21]:
def merge_similar_columns2(fuzzy_match, col_name):
    boolean_mask = data[fuzzy_match].notnull()
    new_values = data[boolean_mask][fuzzy_match].to_numpy()
    indices_fuzzy_matches = data.index[boolean_mask].tolist()

    current_values = data[col_name].to_numpy()
    np.put(current_values, indices_fuzzy_matches, new_values)
    # data_copy.drop(fuzzy_match, axis=1, inplace=True)

In [22]:
%%time
similar_values.apply(lambda x: merge_similar_columns2(x['fuzzy_match'], x['col_name']), axis=1)
# data_copy.loc[(data_copy['pramga'] == 'no-cache') & (data_copy['pragma'] == 'no-cache')][['pragma', 'pramga']]
# data_copy.loc[(data_copy['pragma'] == 'no-cache')]['pragma']

In [24]:
columns_to_remove = list(set(similar_values.fuzzy_match.values.tolist()))
data.drop(columns_to_remove, axis=1, inplace=True)

### Exploration

In [62]:
%matplotlib inline
plt.figure(figsize=(20, 10))

ct_values = data[["content-type", "tracker"]].dropna()

ct_values

data[['content-type', 'tracker']].value_counts().head(20)

In [64]:
%matplotlib inline
plt.figure(figsize=(20, 10))
cl_values = data[["content-length", "tracker"]].dropna().astype('Int32')
sns.histplot(cl_values[cl_values['content-length'] < 1000], bins=10, x="content-length", hue='tracker', element='step')

sns.displot(cl_values[cl_values['content-length'] < 1000], x="content-length", kind='ecdf', hue='tracker')
plt.show()

In [138]:
match2[match2.col_name == "content-length"]

### Explore Distributions

In [580]:
relative_na_per_col = pd.DataFrame((data.isna().sum() / len(data)), columns=["summary"])

plt.figure(figsize=(14, 7))
plt.hist(relative_na_per_col["summary"], log=True, bins=20)
plt.xticks(np.arange(0, 1.1, 0.1))
plt.xlabel("Relative Amount of NAs Per Column", fontdict={"fontsize": 16})
plt.title("Distribution of #NAs Across All Columns", fontsize=16, fontweight="bold")

In [579]:
relative_values_per_col = pd.DataFrame((data.count() / len(data)), columns=["summary"])

plt.figure(figsize=(14, 7))
plt.hist(relative_values_per_col["summary"], log=True, bins=20)
plt.xticks(np.arange(0, 1.1, 0.1))
plt.xlabel("Relative Amount of Values (Non NA) Per Column", fontdict={"fontsize": 16})
plt.title("Distribution of #Values Across All Columns", fontsize=16, fontweight="bold")

In [139]:
rel_frequency_per_col = (
    data.groupby("tracker").count() / data.groupby("tracker").count().sum()
)
rel_frequency_per_col.transpose()

In [111]:
columns_with_more_trackers = rel_frequency_per_col.iloc[
    :, np.where([rel_frequency_per_col.iloc[1, :] > 0.5])[1]
].transpose()
columns_with_more_trackers["total_frequency"] = (
    data[columns_with_more_trackers.index.values].count().values
)
columns_with_more_trackers["total_nas"] = (
    data[columns_with_more_trackers.index.values].isna().sum().values
)
columns_with_more_trackers["ratio_value_total"] = columns_with_more_trackers[
    "total_frequency"
] / len(data)

columns_with_more_trackers.sort_values(by=["total_frequency"], ascending=False)

In [112]:
columns_with_more_non_trackers = rel_frequency_per_col.iloc[
    :, np.where([rel_frequency_per_col.iloc[0, :] > 0.5])[1]
].transpose()
columns_with_more_non_trackers["total_frequency"] = (
    data[columns_with_more_non_trackers.index.values].count().values
)
columns_with_more_non_trackers["total_nas"] = (
    data[columns_with_more_non_trackers.index.values].isna().sum().values
)
columns_with_more_non_trackers["ratio_value_total"] = columns_with_more_non_trackers[
    "total_frequency"
] / len(data)

columns_with_more_non_trackers.sort_values(by=["total_frequency"], ascending=False)

In [115]:
plt.figure(figsize=(20, 7))
sns.kdeplot(columns_with_more_trackers.iloc[:, 1])
sns.kdeplot(columns_with_more_non_trackers.iloc[:, 0])
plt.title(
    "Comparison of Tracker and Non-Tracker Majority within Headers",
    fontdict={"fontsize": 16},
)
plt.xticks(fontsize=14, fontweight="bold", ha="right")

In [577]:
fig, ax = plt.subplots(2, sharex=True, figsize=(14, 7))

ax[0].hist(rel_frequency_per_col.loc[1, :], bins=20, color="tab:red")
ax[1].hist(rel_frequency_per_col.loc[0, :], bins=20)
plt.xticks(np.arange(0, 1.1, 0.1))
fig.suptitle(
    "Relative Frequency per Column by Tracker Classification",
    fontsize=16,
    fontweight="bold",
)

In [568]:
tracker_values = pd.DataFrame(rel_frequency_per_col.loc[1, :])
tracker_values = tracker_values[tracker_values[1] >= 0.95]
tracker_values.rename(columns={1: "rel_frequency"}, inplace=True)
tracker_values

In [576]:
plt.figure(figsize=(20, 7))
plt.plot(
    (
        data["content-type"].value_counts().cumsum()
        / data["content-type"].value_counts().sum()
    )[0:19]
)
plt.title(
    "Cumulative Distribution of Content-Type Header", fontsize=16, fontweight="bold"
)
plt.xticks(rotation=45, fontsize=14, fontweight="bold", ha="right")

In [21]:
def create_value_comparison(header_field, n_values):
    header_field_df = pd.DataFrame(
    data[[f"{header_field}", "tracker"]].value_counts()
        ).reset_index()
    header_field_df.columns = [f"{header_field}", "tracker", "frequency"]
    header_field_df.sort_values(by=["frequency"], ascending=False, inplace=True)

    test_values = header_field_df[0:n_values][f"{header_field}"].values.tolist()
    frequent_content_types = (
        header_field_df[header_field_df[f"{header_field}"].isin(test_values)]
        .reset_index()
        .sort_values(by=f"{header_field}")
    )
    non_tracker = frequent_content_types[frequent_content_types.tracker == 0]
    tracker = frequent_content_types[frequent_content_types.tracker == 1]
    return non_tracker, tracker

In [22]:
def create_value_comparison_plot(non_tracker, tracker, n, header_field):
    r = np.arange(n)
    width = 0.25

    plt.figure(figsize=(20, 7))
    plt.bar(r, non_tracker["frequency"], log=True, label="Non Tracker", width=width)
    plt.bar(r + width, tracker["frequency"], log=True, label="Tracker", width=width)
    plt.xticks(
        r + width / 2,
        non_tracker[f"{header_field}"].values.tolist(),
        rotation=45,
        fontsize=12,
        fontweight="bold",
        ha="right",
    )
    plt.legend()

In [77]:
non_tracker_ct, tracker_ct = create_value_comparison("content-type", 34)
create_value_comparison_plot(non_tracker_ct, tracker_ct, len(tracker_ct))

In [105]:
non_tracker_cl, tracker_cl = create_value_comparison("content-length", 14)
create_value_comparison_plot(non_tracker_cl, tracker_cl, len(tracker_cl), 'content-length')

In [143]:
plt.figure(figsize=(20, 7))
plt.plot(
    (
        data["content-length"].value_counts().cumsum()
        / data["content-length"].value_counts().sum()
    )[0:40]
)
plt.title(
    "Cumulative Distribution of Content-Length Header", fontsize=16, fontweight="bold"
)
plt.xticks(rotation=45, fontsize=14, fontweight="bold", ha="right")

In [122]:
non_tracker_xss, tracker_xss = create_value_comparison("x-xss-protection", 6)
create_value_comparison_plot(non_tracker_xss, tracker_xss, len(tracker_xss), 'x-xss-protection')

In [124]:
non_tracker_cd, tracker_cd = create_value_comparison("content-disposition", 4)
create_value_comparison_plot(non_tracker_cd, tracker_cd, len(tracker_cd), 'content-disposition')

In [125]:
non_tracker_p, tracker_p = create_value_comparison("pragma", 4)
create_value_comparison_plot(non_tracker_p, tracker_p, len(tracker_p), 'pragma')

In [95]:
non_tracker_cl

In [96]:
tracker_cl

In [97]:
pd.DataFrame(
    data[[f"server", "tracker"]].value_counts()
        ).reset_index()

In [23]:
plt.figure(figsize=(20, 7))
# TODO change var name, a bit misleading here
na_per_row = pd.DataFrame((data.isnull().sum(axis=1) - len(data.columns))*-1)
na_per_row["tracker"] = data["tracker"]

sns.histplot(na_per_row, hue='tracker', x=na_per_row.columns.values[0])
sns.displot(na_per_row, hue='tracker', x=na_per_row.columns.values[0], kind='ecdf')

In [24]:
sns.boxplot(data=na_per_row, y=na_per_row.columns.values[0], x='tracker')
sns.displot(na_per_row, hue='tracker', x=na_per_row.columns.values[0], kind='kde', fill=True)

In [371]:
less_ten_headers = data[((data.isnull().sum(axis=1) - len(data.columns))*-1) <= 10].dropna(axis=1, how='all')

less_ten_headers

In [226]:
%%time
data['content-length'].sample(10000, random_state=10).astype('Int64')

In [63]:
columns_as_category = {i: 'category' for i in data.columns.values[:-2]}

In [64]:
columns_as_category

In [65]:
column_test = data.columns.values[6:-2].tolist()


def test_new_categories_update(element):
    categories = data[element].astype("category").cat.categories.values.tolist()
    try:
        np.array(categories, dtype='int64')
        return {element: "Int64"}
    except (ValueError, OverflowError):
        return None

In [66]:
%%time
braze3 = [test_new_categories_update(element) for element in column_test]

In [67]:
braze3 = list(filter(lambda x: type(x) is dict, braze3))
braze3 = {k: v for d in braze3 for k, v in d.items()}
braze3

In [80]:
data['x-goog-generation'].astype('Int64').max()

In [68]:
columns_as_category.update(braze3)
columns_as_category

In [16]:
del columns_as_category["query"]
del columns_as_category["protocol"]
data.drop(['protocol', 'query'], axis=1, inplace=True)

In [17]:
data = data.astype(columns_as_category)

In [12]:
data.dtypes

In [93]:
def variance_per_column(column):
    unique_values = int(len(set(data[column]))) - 1
    na_values = data[column].isna().sum()
    rows_with_values = data[column].value_counts().sum()
    ratio_real_values =  round(rows_with_values / unique_values, 3)
    tracker_ratio = data[data[column].notnull()].tracker.value_counts()
    try:
        trackers = tracker_ratio[1]
    except KeyError:
        trackers = 0
    try:
        non_trackers = tracker_ratio[0]
    except KeyError:
        non_trackers = 0
    return [column, unique_values, rows_with_values,
            na_values, round(na_values / len(data), 3), round(rows_with_values/ len(data), 3),
            ratio_real_values, trackers, non_trackers]

In [34]:
def reduced_variance_per_column(column):
    unique_values = int(len(set(data[column]))) - 1
    na_values = data[column].isna().sum()
    return [column, unique_values, round(na_values / len(data), 3)]

In [150]:
%%time
number_of_elements = np.array([variance_per_column(column) for column in data.iloc[:, 4:].columns])

In [35]:
number_of_elements_reduced = np.array([reduced_variance_per_column(column) for column in data.iloc[:, 4:-2].columns])

In [36]:
summary_table = pd.DataFrame(number_of_elements_reduced, columns=["header_name",
                                                          "unique_values",
                                                          "na_ratio"])
summary_table['unique_values'] = summary_table['unique_values'].astype("Int32")
summary_table["na_ratio"] = summary_table["na_ratio"].astype("float32")

In [152]:
summary_table = pd.DataFrame(number_of_elements, columns=['header_name', 'unique_values',
                                                          'non_nas', 'nas', 'na_ratio',
                                                          'non_na_ratio', 'ratio_real_values',
                                                          'trackers', 'non_trackers'])

summary_table['unique_values'] = summary_table['unique_values'].astype("Int32")
summary_table['non_nas'] = summary_table['non_nas'].astype("Int32")
summary_table['nas'] = summary_table['nas'].astype("Int32")
summary_table['na_ratio'] = summary_table['na_ratio'].astype("float32")
summary_table['non_na_ratio'] = summary_table['non_na_ratio'].astype("float32")
summary_table['ratio_real_values'] = summary_table['ratio_real_values'].astype("float32")
summary_table['trackers'] = summary_table['trackers'].astype("Int32")
summary_table['non_trackers'] = summary_table['non_trackers'].astype("Int32")

summary_table.sort_values(by='ratio_real_values', ascending=False, inplace=True)
summary_table.reset_index(drop=True, inplace=True)
summary_table.drop([0], inplace=True)
summary_table.reset_index(drop=True, inplace=True)

In [98]:
summary_table

In [142]:
summary_table[summary_table['header_name'] == 'date']

In [37]:
remove_headers_with_one_na_ratio = summary_table[summary_table['na_ratio'] == 1].header_name.values.tolist()

data.drop(remove_headers_with_one_na_ratio, axis=1, inplace=True)

In [38]:
# summary_table[(summary_table['unique_values'] <= 1) & (summary_table['na_ratio'] != 1)].header_name.values.tolist()

remove_headers_with_one_value = summary_table[(summary_table['unique_values'] <= 1) & (summary_table['na_ratio'] != 1)].header_name.values.tolist()

In [39]:
data.drop(remove_headers_with_one_value, axis=1, inplace=True)

In [40]:
data

In [1112]:
sns.boxplot(data=data, x='fastly-restarts')

In [41]:
data.info()

### Check ID Headers

In [261]:
%%time
# [print(col) for col in summary_table.header_name.values if '-id' in col]
id_headers = list(filter(lambda y: y != False, map(lambda x: x if x[-2:] == 'id' else False, summary_table.header_name
                                            .values)))

In [262]:
summary_table[summary_table['header_name'].isin(id_headers)]

#### Keep relevant columns

In [42]:
def variance_per_column_2(column):
    tracker_ratio = data[data[column].notnull()].tracker.value_counts()
    try:
        trackers = tracker_ratio[1]
    except KeyError:
        trackers = 0
    try:
        non_trackers = tracker_ratio[0]
    except KeyError:
        non_trackers = 0
    return [column, trackers, non_trackers]

In [43]:
number_of_elements_reduced = np.array([variance_per_column_2(column) for column in data.iloc[:, 4:-2].columns])
summary_table = pd.DataFrame(number_of_elements_reduced, columns=["header_name",
                                                                  "trackers",
                                                                  "non_trackers"])
summary_table['trackers'] = summary_table['trackers'].astype("Int32")
summary_table["non_trackers"] = summary_table["non_trackers"].astype("float32")
summary_table['ratio'] = (summary_table['trackers']/summary_table['non_trackers'])*100
summary_table['ratio2'] = (summary_table['non_trackers']/summary_table['trackers'])*100

In [44]:
summary_table

In [45]:
only_non_tracker_col = summary_table[summary_table['ratio'] <= 10].header_name.values.tolist()

only_tracker_col = summary_table[summary_table['ratio2'] <= 10].header_name.values.tolist()

In [46]:
summary_table[summary_table['ratio'] <= 10]

In [64]:
only_tracker_col

In [77]:
data[(data['google-creative-id'].notnull()) & (data['google-lineitem-id'].notnull())][['google-lineitem-id', 'google-creative-id']]

# CHANGE COMBINATION TO +1 for each col in col_list

In [117]:
test_dict = {0: 2, 1:3}

test_dict

In [78]:
data[(data['google-creative-id'].notnull()) & (data['tracker'] == 1)].index.tolist()

In [101]:
def concise_information(col_list, classification, dataset):
    indices = list()

    for col in col_list:
        indices.append(dataset[(dataset[col].notnull()) & (dataset['tracker'] == classification)].index.tolist())

    return indices

In [48]:
data['comb_col_non_tracker'] = 0
data['comb_col_tracker'] = 0

In [102]:
from collections import Counter

tests = dict(Counter(list(np.concatenate(concise_information(only_tracker_col, 1, data)).flat)))
tests

In [103]:
for key, value in tests.items():
    data.at[key, "comb_col_tracker"] = value

In [105]:
data[data['comb_col_tracker'] > 3]

In [110]:
%%time
for idx in concise_information(only_tracker_col, 1):
    data.at[idx, 'comb_col_tracker'] = 1

In [111]:
%%time
for idx in concise_information(only_non_tracker_col, 0):
    data.at[idx, 'comb_col_non_tracker'] = 1

In [112]:
data.drop(only_non_tracker_col, axis=1, inplace=True)
data.drop(only_tracker_col, axis=1, inplace=True)

In [113]:
data

In [96]:
def impute_value(element):
    if data[element].dtype == 'category':
        data[element] = data[element].cat.add_categories('missing')
        data[element].fillna('missing', inplace=True)
    else:
        data[element].fillna(91219942022, inplace=True)

In [ ]:
[impute_value(value) for value in data.columns.values]

#### Outlier detection

In [2]:
list_of_categorical_cols = list(data.iloc[:, 4:-4].select_dtypes('category').columns.values
                                                      .tolist())

list_of_integer_cols = list(data.iloc[:, 4:-4].select_dtypes('Int64').columns.values
                                                      .tolist())

In [1]:
data.info()

In [133]:
number_of_trackers = len(data[data['tracker'] ==1])
number_of_non_trackers = len(data[data['tracker'] == 0])

In [134]:
summary_table['ratio_tracker'] = summary_table['trackers'] / number_of_trackers
summary_table['ratio_non_tracker'] = summary_table['non_trackers'] / number_of_non_trackers

In [137]:
summary_table

In [266]:
def impute_value(element, classification):
    if element in list_of_integer_cols:
        # print(element, classification)
        data.loc[data['tracker'] == classification, element]\
            .fillna(data[data['tracker']==classification][element].median())

    if element in list_of_categorical_cols:
        print(element, classification)
        data.loc[data['tracker'] == classification, element] = data.loc[data['tracker'] == classification, element]\
            .fillna(data[data['tracker']==classification][element].mode().iloc[0])

In [245]:
impute_col_list_t = summary_table[summary_table['ratio_tracker'] < 0.4].header_name.values.tolist()
impute_col_list_nt = summary_table[summary_table['ratio_non_tracker'] < 0.4].header_name.values.tolist()

In [256]:
for header in impute_col_list_t:
    impute_value(header, 0)

for header in impute_col_list_t:
    impute_value(header, 1)

for header in impute_col_list_nt:
    impute_value(header, 1)

for header in impute_col_list_nt:
    impute_value(header, 0)

In [253]:
impute_col_list_t

In [259]:
data[['x-xss-protection', 'tracker']]

In [269]:
impute_value('x-xss-protection', 1)

In [270]:
data[['x-xss-protection', 'tracker']]

In [63]:
data